In [1]:
from Acesso import Login
from Query import Query
from Moeda import Moeda
import pandas as pd

pd.set_option('display.max_columns',None)

s=Login()

sql=Query(s.usuario,s.senha,s.database,s.server)

querys={
    
    'Faltas':"""
        
    IF MONTH(GETDATE())=1

        SELECT * FROM netfeira.vw_falta
        WHERE YEAR([Data de Falta])=YEAR(GETDATE())-1 AND MONTH([Data de Falta])=12
        
    ELSE

        IF DAY(GETDATE())=1

            SELECT * FROM netfeira.vw_falta
            WHERE YEAR([Data de Falta])=YEAR(GETDATE()) AND MONTH([Data de Falta])=MONTH(GETDATE())-1

        ELSE
                
            SELECT * FROM netfeira.vw_falta
            WHERE [Data de Falta]=CONVERT(DATETIME,CAST(GETDATE() AS DATE),101) 
    
    """,
    
    'Vendedores':"""
    
    SELECT vend.[ID Vendedor],vend.[Nome Resumido],
	vend.[ID Equipe],vend.DDD,vend.Telefone
	FROM netfeira.vw_vendedor vend
    WHERE Categoria='CLT' AND vend.[Status do Vendedor]='ATIVO' AND vend.Telefone IS NOT NULL
    
    
    """,
    
    'Supervisor':"""
    
    SELECT * FROM netfeira.vw_supervisor
    
    """
    
}

In [2]:
tabelas_df=sql.CriarTabela(kwargs=querys)

In [3]:
tabelas_df['Faltas'].columns

Index(['Data de Falta', 'ID Vendedor', 'ID Cliente', 'Nome Fantasia', 'Pedido',
       'SKU', 'Produto', 'Unid. VDA', 'Qtde. VDA', 'Valor Unitário',
       'Total do Pedido'],
      dtype='object')

In [4]:
tabelas_df['Vendedores'].columns

Index(['ID Vendedor', 'Nome Resumido', 'ID Equipe', 'DDD', 'Telefone'], dtype='object')

In [5]:
tabelas_df['Supervisor'].columns

Index(['ID Equipe', 'Equipe', 'ID Sup', 'Supervisor', 'Email Sup', 'DDD Sup',
       'Telefone Sup', 'ID Gerente', 'Gerente', 'Email Gerente', 'DDD Gerente',
       'Telefone Gerente'],
      dtype='object')

In [6]:
tabelas_df['Vendedores']=tabelas_df['Vendedores'].merge(tabelas_df['Supervisor'],on='ID Equipe',how='inner')

In [7]:
tabelas_df['Faltas']=tabelas_df['Faltas'].merge(tabelas_df['Vendedores'],on='ID Vendedor',how='inner')

In [8]:
tabelas_df['Consolidado']=tabelas_df['Faltas'][['ID Vendedor','Nome Resumido','Equipe','Supervisor','DDD','Telefone','Total do Pedido']].groupby(['ID Vendedor','Nome Resumido','Equipe','Supervisor','DDD','Telefone'],as_index=False).sum()

In [9]:
tabelas_df['Consolidado'].sort_values('Total do Pedido',ascending=False,ignore_index=True,inplace=True)

In [10]:
tabelas_df['Consolidado']['Pedidos']=tabelas_df['Consolidado']['ID Vendedor'].apply(lambda info: len(tabelas_df['Faltas']['Pedido'].loc[tabelas_df['Faltas']['ID Vendedor']==info].unique().tolist()))

In [11]:
tabelas_df['Consolidado']['MIX']=tabelas_df['Consolidado']['ID Vendedor'].apply(lambda info: len(tabelas_df['Faltas']['SKU'].loc[tabelas_df['Faltas']['ID Vendedor']==info].unique().tolist()))

In [12]:
codigos=tabelas_df['Consolidado']['ID Vendedor'].tolist()

temp=[]

for c in codigos:
    
    df=pd.DataFrame()
    
    df=tabelas_df['Consolidado'].loc[tabelas_df['Consolidado']['ID Vendedor']==c]
    
    vendedor=df['Nome Resumido'].tolist()
           
    vendedor=str(vendedor[-1]).title()
    
    total=df['Total do Pedido'].sum()
    
    total=Moeda.FormatarMoeda(total)
    
    mensagem=f"""
    
    Bom dia;
    
    {vendedor} tudo bem, estou aqui para te informar que identificamos que houve corte no valor de R$ {total}.
        
    
    """
    
    temp.append(mensagem)
    
    pass


tabelas_df['Consolidado']['Mensagens']=temp


In [13]:
def Telefone(num):
    
    num=str(num)
    
    num=num.split('.')
    
    return num[0]
    
    pass

In [14]:
tabelas_df['Consolidado']['DDD']=tabelas_df['Consolidado']['DDD'].apply(Telefone)

In [15]:
tabelas_df['Consolidado']['Telefone']=tabelas_df['Consolidado']['Telefone'].apply(Telefone)

In [16]:
tabelas_df['Faltas']=tabelas_df['Faltas'].merge(tabelas_df['Cliente'],on='ID Cliente',how='inner')

KeyError: 'Cliente'

In [ ]:
tabelas_df['Vendedores'].columns

In [17]:
tabelas_df['Faltas'].columns

Index(['Data de Falta', 'ID Vendedor', 'ID Cliente', 'Nome Fantasia', 'Pedido',
       'SKU', 'Produto', 'Unid. VDA', 'Qtde. VDA', 'Valor Unitário',
       'Total do Pedido', 'Nome Resumido', 'ID Equipe', 'DDD', 'Telefone',
       'Equipe', 'ID Sup', 'Supervisor', 'Email Sup', 'DDD Sup',
       'Telefone Sup', 'ID Gerente', 'Gerente', 'Email Gerente', 'DDD Gerente',
       'Telefone Gerente'],
      dtype='object')